In [1]:
import rasterio
import dask.array as da
from dask_rasterio import read_raster, write_raster
import os
import fnmatch
import numpy as np
import geopandas as gpd
import shapely
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy.spatial as es
import seaborn as sns
import matplotlib.pyplot as plt
import earthpy.plot as ep
import imageio
import rasterstats

In [25]:
input_dir = r"Z:\temp\storms\Hurricane Katrina 2005"
outputDir= r'Z:\temp\!Accumulated'
cropped_dir = os.path.join(input_dir, 'cropped')
img_dir = os.path.join(cropped_dir, 'img')
outputFilename = input_dir.split('\\')[-1]+'-Inc-Cropped.gif'
storm = input_dir.split('\\')[-1]
statsPlotOutput_fn = input_dir.split('\\')[-1]+'-stats.png'
sns.set_style("white")
sns.set(font_scale=1.5)

isExist_img = os.path.exists(img_dir)
if not isExist_img:
    os.makedirs(img_dir)

In [26]:
merge_dir=[]
for filename in fnmatch.filter(os.listdir(cropped_dir),'*.01h'): 
    merge_dir.append((os.path.join(cropped_dir, filename)))

merge_dir.sort()

In [23]:
arrayList = []
rasterStats = {}
for raster in merge_dir:
        # print(f'adding {raster} to dask array list: map2array')
        src = rasterio.open(raster)
        array = src.read(1)
        # array[array==0] = np.nan    
        array[array==9999] = np.nan
        arrayList.append(array)
        
        # Get Rasterstats
        stats = rasterstats.zonal_stats(r"Z:\GIS\StageIv Boundary.shp", raster, stats=['min', 'max','mean', 'count', 'nodata'])
        percentMissing = stats[0]['nodata']/stats[0]['count']
        percentMissing = round(percentMissing * 100, 2)
        # print ('\n'+raster.split('//')[-1] + ' '+ str(percentMissing))
        stats[0]['percentMissing'] = percentMissing
        rasterStats[raster] = stats
        src.close()
rasterStats

{}

In [20]:
# Get the biggest max for the plot colorBar maxValue to be.
# Record for which Hours that have a missing data covering more than 3% of the raster area.
# Make list of the percent Area missing for each Hour that has missing data
hasMissingHoursIntegerList = []
percentAreaMissingList = []
bigMax = 0
hour = 0
summaryTable = [{}]
summaryTableColumnKeys = ['Event', 'Total Duration (hr)', 'Hours With Missing Data', '%% of Duration With Missing Data' ]
for raster in rasterStats:
    hour +=1
    max = rasterStats[raster][0]['max']
    if (max > bigMax):
        bigMax = max
    # If over 3% of raster is Nan, count it as having missing data.
    if (rasterStats[raster][0]['percentMissing'] > 3.0):
        hasMissingHoursIntegerList.append(hour)
        percentAreaMissingList.append(rasterStats[raster][0]['percentMissing'])
hoursTotal = hour
hoursMissingTotal = len(hasMissingHoursIntegerList)
percentHoursMissing = round(hoursMissingTotal/hoursTotal, 2)* 100

# rasterStats

In [10]:
hour = 0
for i, array in enumerate(arrayList):
    hour += 1
    title = merge_dir[i].split('\\')[-1]
    img_filename = img_dir + f'\\{title}.png'
    crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
    fig, ax = plt.subplots(figsize=(20, 15))
    la = gpd.read_file("Z:\GIS\Louisiana.shp")
    bbox = la.total_bounds
    la_extent=bbox[[0,2,1,3]]
    bbox_lwi = crop_shp.total_bounds
    lwi_extent = bbox_lwi[[0,2,1,3]]
    array[array==9999] = np.nan
    im = plt.imshow(array, extent=lwi_extent, cmap='rainbow', vmin=0, vmax=bigMax)
    cb = plt.colorbar(im, shrink=.6)
    ax.set(title=f"{storm}\n{title} Incremental Precip (mm), Total Duration (hr): {hoursTotal} \nHour:{hour}, %Hours with Missing Data: {percentHoursMissing}%")
    ax.set_axis_off()
    la.boundary.plot(ax=plt.gca(), color='darkgrey')
    plt.savefig((img_filename))
    plt.close()


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  from ipykernel import kernelapp as app
C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  from ipykernel import kernelapp as app
C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  from ipykernel import kernelapp as app
C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:15: MatplotlibDeprecationWarning: Auto-remo

In [114]:
print (f'Creating gif movie: {outputFilename}')
imgList=[]
for img_file in fnmatch.filter(os.listdir(img_dir),'*.png'): 
    imgList.append((os.path.join(img_dir, img_file)))
imgList.sort()

imagesArray = []
for imageFile in imgList:
    # print(f'adding {imageFile} to imagesArray.')
    imagesArray.append(imageio.imread(imageFile))
imageio.mimsave(os.path.join(outputDir, outputFilename), imagesArray, duration=0.1)
print ('<----- Done. ----->\n')

Creating gif movie: Hurricane Katrina 2005-Inc-Cropped.gif
<----- Done. ----->



In [11]:
# Set up Data arrays for Stats Timeseries Plots
x_arr = np.arange(1, hoursTotal)
y_MissingData=[]
y_PercentAreaMissing = []
y=0
for hr in x_arr:
    if hr in hasMissingHoursIntegerList:
        y+=1
        i = hasMissingHoursIntegerList.index(hr)  
        y_PercentAreaMissing.append(percentAreaMissingList[i])
    else:
        y_PercentAreaMissing.append(0)   
    y_MissingData.append(y)
    
y_arr_MissingData = np.array(y_MissingData)
y_arr_PercentAreaMissing = np.array(y_PercentAreaMissing)
fig, ax = plt.subplots(figsize=(20, 10))

# Hourly Datasets with Missing Values Plot
plt.subplot(121)
plt.suptitle(f'{storm}')
plt.plot(x_arr,y_arr_MissingData)
plt.ylabel('Hourly Datasets with Missing Values')
plt.xlabel('Time (hr)')

# Percent Area of Missing Data Plot
plt.subplot(122)
plt.plot(x_arr,y_arr_PercentAreaMissing)
plt.ylabel('Percent Area Missing')
plt.xlabel('Time (hr)')
# plt.show()
plt.savefig(os.path.join(outputDir, statsPlotOutput_fn))
plt.close()

